In [2]:
from functools import lru_cache
from natasha import MorphVocab, AddrExtractor
import pandas as pd

In [5]:
df = pd.read_clipboard(sep="\t", index_col=0)

In [6]:
df

,пол,Регион,Страна,Город,Область,a,b
1,ж,"Россия, Томская область, Томск",Россия,Томск,Томская область,True,Областной центр
2,ж,"Россия, Москва",Россия,Москва,Москва,True,Областной центр
3,м,Не указано,NaN,Не указано,NaN,False,Районный город
4,м,Не указано,NaN,Не указано,NaN,False,Районный город
5,м,"Россия, Самарская область, Самара",Россия,Самара,Самарская область,True,Областной центр
6,NaN,"Россия, Пермский край, Пермь",Россия,Пермь,Пермский край,True,Областной центр
7,ж,Не указано,NaN,Не указано,NaN,False,Районный город
8,м,"Россия, Татарстан, Казань",Россия,Казань,Татарстан,False,Районный город
9,м,"Россия, Нижегородская область, Дзержинск",Россия,Дзержинск,Нижегородская область,False,Районный город


In [8]:
morph_vocab = MorphVocab()
addr_extractor = AddrExtractor(morph=morph_vocab)
@lru_cache(maxsize=2048)
def parse_addr(addr_str, addr_extractor=addr_extractor):
    ext = addr_extractor.find(addr_str)
    if not ext:
        return pd.Series(dtype=str)
    res = pd.Series(
        {part.type if part.type else "город": part.value 
         for part in ext.fact.parts},
        dtype=str
    )
    return res

In [9]:
res = df.join(df["Регион"].apply(parse_addr))

In [10]:
res

,пол,Регион,Страна,Город,Область,a,b,страна,область,город,край
1,ж,"Россия, Томская область, Томск",Россия,Томск,Томская область,True,Областной центр,Россия,Томская,Томск,NaN
2,ж,"Россия, Москва",Россия,Москва,Москва,True,Областной центр,Россия,NaN,Москва,NaN
3,м,Не указано,NaN,Не указано,NaN,False,Районный город,NaN,NaN,NaN,NaN
4,м,Не указано,NaN,Не указано,NaN,False,Районный город,NaN,NaN,NaN,NaN
5,м,"Россия, Самарская область, Самара",Россия,Самара,Самарская область,True,Областной центр,Россия,Самарская,Самара,NaN
6,NaN,"Россия, Пермский край, Пермь",Россия,Пермь,Пермский край,True,Областной центр,Россия,NaN,Пермь,Пермский
7,ж,Не указано,NaN,Не указано,NaN,False,Районный город,NaN,NaN,NaN,NaN
8,м,"Россия, Татарстан, Казань",Россия,Казань,Татарстан,False,Районный город,Россия,NaN,Казань,NaN
9,м,"Россия, Нижегородская область, Дзержинск",Россия,Дзержинск,Нижегородская область,False,Районный город,Россия,Нижегородская,Дзержинск,NaN
